# Part 2 - 3D mesh tallies

Mesh tallies can also be used to visualise neutron interactions spatially throughout a geometry in 3D.

This notebook allows users to create a simple geometry from a few different materials and plot the results of a 3D regular mesh tally applied to the geometry.

This first code block defines the model geometry, materials and neutron source.

In [ ]:
import openmc
import os

# MATERIALS

breeder_material = openmc.Material(1, "PbLi")   # Pb84.2Li15.8
breeder_material.add_element('Pb', 84.2, percent_type='ao')
breeder_material.add_element('Li', 15.8, percent_type='ao', enrichment=7.0, enrichment_target='Li6', enrichment_type='ao')   # natural enrichment = 7% Li6
breeder_material.set_density('atom/b-cm', 3.2720171e-2)   # around 11 g/cm3

copper = openmc.Material(name='Copper')
copper.set_density('g/cm3', 8.5)
copper.add_element('Cu', 1.0)

eurofer = openmc.Material(name='iron')
eurofer.set_density('g/cm3', 7.75)
eurofer.add_element('Fe', 89.067, percent_type='wo')

mats = openmc.Materials([breeder_material, eurofer, copper])


# GEOMETRY

# surfaces
central_sol_surface = openmc.ZCylinder(r=100)
central_shield_outer_surface = openmc.ZCylinder(r=110)
vessel_inner_surface = openmc.Sphere(r=500)
first_wall_outer_surface = openmc.Sphere(r=510)
breeder_blanket_outer_surface = openmc.Sphere(r=610, boundary_type='vacuum')

# cells
central_sol_region = -central_sol_surface & -breeder_blanket_outer_surface
central_sol_cell = openmc.Cell(region=central_sol_region)
central_sol_cell.fill = copper

central_shield_region = +central_sol_surface & -central_shield_outer_surface & -breeder_blanket_outer_surface
central_shield_cell = openmc.Cell(region=central_shield_region)
central_shield_cell.fill = eurofer

inner_vessel_region = -vessel_inner_surface & +central_shield_outer_surface
inner_vessel_cell = openmc.Cell(region=inner_vessel_region)
# no material set as default is vacuum

first_wall_region = -first_wall_outer_surface & +vessel_inner_surface
first_wall_cell = openmc.Cell(region=first_wall_region)
first_wall_cell.fill = eurofer

breeder_blanket_region = +first_wall_outer_surface & -breeder_blanket_outer_surface & +central_shield_outer_surface
breeder_blanket_cell = openmc.Cell(region=breeder_blanket_region)
breeder_blanket_cell.fill = breeder_material

universe = openmc.Universe(cells=[central_sol_cell, central_shield_cell, inner_vessel_cell, first_wall_cell, breeder_blanket_cell])
geom = openmc.Geometry(universe)


# SIMULATION SETTINGS

# Instantiate a Settings object
sett = openmc.Settings()
batches = 2
sett.batches = batches
sett.inactive = 0
sett.particles = 5000
sett.run_mode = 'fixed source'

# Create a DT point source
source = openmc.Source()
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
source.space = openmc.stats.Point((150, 150, 0))

sett.source = source

This code block creates a 3D regular mesh between two coordinates with a specified resolution in each axis.

In [ ]:
mesh = openmc.RegularMesh()

mesh.dimension = [100, 50, 100]  # number of mesh cells in each dimension (resolution)

mesh.lower_left = [-750, 0, -750]  # x,y,z coordinates
mesh.upper_right = [750, 750, 750]  # x,y,z coordinates

This code block creates two tallies on the mesh to record heating and tritium production.

In [ ]:
tallies = openmc.Tallies()
# Create mesh filter for tally
mesh_filter = openmc.MeshFilter(mesh)

# Create flux mesh tally to score flux
mesh_tally = openmc.Tally(tally_id=1, name='tbr_on_mesh')  # note the tally_id is specified
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['(n,Xt)']
tallies.append(mesh_tally)

# Create flux mesh tally to score flux
mesh_tally = openmc.Tally(tally_id=2, name='heating_on_mesh')  # note the tally_id is specified
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['heating']
tallies.append(mesh_tally)

This next code block performs the simulation.

In [ ]:
# deletes old statepoint and summary files
!rm s*.h5

# Run OpenMC!
model = openmc.model.Model(geom, mats, sett, tallies)
sp_filename = model.run()

This code block runs a python function which extracts the mesh tally data from the statepoint.h5 file and saves it as a vtk file.

In [ ]:

# loads up the output file from the simulation
statepoint = openmc.StatePoint('statepoint.2.h5')

# extracts the mesh tally by name
my_tbr_tally = statepoint.get_tally(name='tbr_on_mesh')

# converts the tally result into a VTK file
mesh.write_data_to_vtk(
    filename="tbr_tally_on_mesh.vtk",
    datasets={"mean": my_tbr_tally.mean}  # the first "mean" is the name of the data set label inside the vtk file
)

# extracts the mesh tally by name
my_heating_tally = statepoint.get_tally(name='heating_on_mesh')

# converts the tally result into a VTK file
# this time stadnard deviation error on the tally is added to the vtk file as another data set
# the tally is also scaled from eV per source particle to Joules per source particle 1eV = 1.60218e-19J)
# Try adding another scaling term to multiplying by the number of neutrons emitted per second would which would convert the units to Watts
mesh.write_data_to_vtk(
    filename="heating_tally_on_mesh.vtk",
    datasets={"mean": my_heating_tally.mean * 1.60218e-19, "std_dev": my_heating_tally.std_dev}
)

Click on the newly created vtk file in the file explorer to the left and download the vtk files onto your base computer and open them with a VTK file reader such as Paraview or Visit.

Paraview can be downloaded here: https://www.paraview.org/download/.
Visit can be downloaded here: https://wci.llnl.gov/simulation/computer-codes/visit/downloads.

**Learning Outcomes for Part 2:**

- Mesh tallies can be used in neutronics simulations to measure a variety of different reactions such as neutron absorption, tritium production and flux.
- How neutrons are dissipated around the reactor.